# Randomized Benchmarking Overview

### Contributors

Shelly Garion$^{1}$, Yael Ben-Haim$^{2}$ and David McKay$^{2}$

1. IBM Research Haifa, Haifa University Campus, Mount Carmel Haifa, Israel
2. IBM T.J. Watson Research Center, Yorktown Heights, NY, USA

### References

1. Easwar Magesan, J. M. Gambetta, and Joseph Emerson, Robust randomized benchmarking of quantum processes,
https://arxiv.org/pdf/1009.3639

2. Easwar Magesan,, Jay M. Gambetta, and Joseph Emerson, Characterizing Quantum Gates via Randomized Benchmarking,
https://arxiv.org/pdf/1109.6887

3. A. D. C'orcoles, Jay M. Gambetta, Jerry M. Chow, John A. Smolin, Matthew Ware, J. D. Strand, B. L. T. Plourde, and M. Steffen, Supplementary material for ''Process verification of two-qubit quantum gates by randomized benchmarking'', https://arxiv.org/pdf/1210.7011

4. Jay M. Gambetta, A. D. C´orcoles, S. T. Merkel, B. R. Johnson, John A. Smolin, Jerry M. Chow,
Colm A. Ryan, Chad Rigetti, S. Poletto, Thomas A. Ohki, Mark B. Ketchen, and M. Steffen,
Characterization of addressability by simultaneous randomized benchmarking, https://arxiv.org/pdf/1204.6308

5. David C. McKay, Sarah Sheldon, John A. Smolin, Jerry M. Chow, and Jay M. Gambetta, Three Qubit Randomized Benchmarking,
https://arxiv.org/pdf/1712.06550

## Intorduction

One of the main challenges in building a quantum information processor is the non-scalability of completely
characterizing the noise affecting a quantum system via process tomography. In addition, process tomography is sensitive to noise in the pre- and post rotation gates plus the measurements (SPAM errors). Gateset tomography can take these errors into account, but the scaling is even worse.  A complete characterization
of the noise is useful because it allows for  the determination of good error-correction schemes, and thus
the possibility of reliable transmission of quantum information.

Since complete process tomography is infeasible for large systems, there is growing interest in scalable
methods for partially characterizing the noise affecting a quantum system. A scalable (in the number $n$ of qubits comprising the system) and robust algorithm for benchmarking the full set of Clifford gates by a single parameter using randomization techniques was presented in [1]. The concept of using randomization methods for benchmarking quantum gates is commonly called **Randomized Benchmarking
(RB)**.

## The Randomized Benchmarking Protocol

A RB protocol consists of the following steps:

(We should first import the relevant qiskit classes for the demonstration).

In [5]:
#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import Qiskit classes classes
import qiskit
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb

### Step 1: Generate RB sequences

The RB sequences consist of random Clifford elements chosen uniformly from the Clifford group on $n$-qubits, 
including a computed reversal element,
that should return the qubits to the initial state.

More precisely, for each length $m$, we choose $K_m$ RB sequences. 
Each such sequence contains $m$ random elements $C_{i_j}$ chosen uniformly from the Clifford group on $n$-qubits, and the $m+1$ element is defined as follows: $C_{i_{m+1}} = (C_{i_1}\cdot ... \cdot C_{i_m})^{-1}$.

For example, we generate below several sequences of 2-qubit Clifford circuits.

In [9]:
#Generate RB circuits (2Q RB)

#number of qubits
nQ=2 
rb_opts = {}
#Number of Cliffords in the sequence
rb_opts['length_vector'] = [1, 10, 20, 50, 75, 100, 125]
#Number of seeds (random sequences)
rb_opts['nseeds'] = 5 
#Default pattern
rb_opts['rb_pattern'] = [[0,1]]

rb_circs, xdata = rb.randomized_benchmarking_seq(**rb_opts)

As an example, we print the circuit corresponding to the first RB sequence

In [11]:
print(rb_circs[0][0])

         ┌───┐┌─────┐┌───┐                      ┌───┐┌───┐┌───┐ ░ ┌───┐┌─────┐»
qr_0: |0>┤ H ├┤ Sdg ├┤ H ├──■───────────────────┤ H ├┤ S ├┤ X ├─░─┤ X ├┤ Sdg ├»
         └───┘└─────┘└───┘┌─┴─┐┌─────┐┌───┐┌───┐└───┘└───┘└───┘ ░ └───┘└─────┘»
qr_1: |0>─────────────────┤ X ├┤ Sdg ├┤ H ├┤ X ├────────────────░─────────────»
                          └───┘└─────┘└───┘└───┘                ░             »
 cr_0: 0 ═════════════════════════════════════════════════════════════════════»
                                                                              »
 cr_1: 0 ═════════════════════════════════════════════════════════════════════»
                                                                              »
«      ┌───┐                       ┌───┐┌───┐┌───┐┌─┐
«qr_0: ┤ H ├─────────────────■─────┤ H ├┤ S ├┤ H ├┤M├
«      └───┘┌───┐┌───┐┌───┐┌─┴─┐┌─┐└───┘└───┘└───┘└╥┘
«qr_1: ─────┤ X ├┤ H ├┤ S ├┤ X ├┤M├────────────────╫─
«           └───┘└───┘└───┘└───┘└╥┘                ║ 
«cr_0: ═══

One can verify that the Unitary representing each RB circuit should be the identity (with a global phase). 
We simulate this using Aer unitary simulator.

In [10]:
#Create a new circuit without the measurement
qc = qiskit.QuantumCircuit(*rb_circs[0][-1].qregs,*rb_circs[0][-1].cregs)
for i in rb_circs[0][-1][0:-nQ]:
    qc._attach(i)

In [12]:
#Create a new circuit without the measurement
qc = qiskit.QuantumCircuit(*rb_circs[0][-1].qregs,*rb_circs[0][-1].cregs)
for i in rb_circs[0][-1][0:-nQ]:
    qc._attach(i)

In [13]:
#The Unitary is an identity (with a global phase)
backend = qiskit.Aer.get_backend('unitary_simulator')
basis_gates = ['u1','u2','u3','cx'] # use U,CX for now
basis_gates_str = ','.join(basis_gates)
job = qiskit.execute(qc, backend=backend, basis_gates=basis_gates_str)
print(np.around(job.result().get_unitary(),3))

[[-0.707-0.707j  0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   -0.j    -0.707-0.707j -0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j    -0.707-0.707j  0.   +0.j   ]
 [ 0.   -0.j     0.   +0.j     0.   +0.j    -0.707-0.707j]]


### Step 2: Execute the RB sequences (with some noise)

We can execute the RB sequences either using Qiskit Aer Simulator (with some noise model) or using IBMQ provider, and obtain a list of results.

By assumption each operation $C_{i_j}$ is allowed to have some error, represnted by $\Lambda_{i_j,j}$, and each sequence can be modeled by the operation:
$$\textit{S}_{\textbf{i}_\textbf{m}} = \bigcirc_{j=1}^{m+1} (\Lambda_{i_j,j} \circ C_{i_j})$$
where ${\textbf{i}_\textbf{m}} = (i_1,...,i_m)$ and $i_{m+1}$ is uniquely determined by ${\textbf{i}_\textbf{m}}$.

In [14]:
# Run on a noisy simulator
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.002, 1), ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(0.002, 2), 'cx')

backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = 'u1,u2,u3,cx'
result_list = []

### Step 3: Get statistics about the survival probabilities

For each of the $K_m$ sequences the survival probability $Tr[E_\psi \textit{S}_{\textbf{i}_\textbf{m}}(\rho_\psi)]$
is measured. 
Here $\rho_\psi$ is the initial state taking into account preparation errors and $E_\psi$ is the
POVM element that takes into account measurement errors.
In the ideal (noise-free) case $\rho_\psi = E_\psi = |\psi \rangle \langle \psi|$. 

In practice one can measure the probability to go back to the exact initial state, i.e. all the qubits in the ground state ($|00...0\rangle$) or just the probability for one of the qubits to return back to the ground state. Measuring the qubits independently can be more convenient if a correlated measurement scheme is not possible. Both measurements will fit to the same decay parameter according to the properties of the twirl. 

### Step 4: Find the averaged sequence fidelity

Average over the $K_m$ random realizations to find the averaged sequence **fidelity**,
$$F_{seq}(m,\psi) = Tr[E_\psi \textit{S}_{K_m}(\rho_\psi)]$$
where 
$$\textit{S}_{K_m} = \frac{1}{K_m} \sum_{\textbf{i}_\textbf{m}} \textit{S}_{\textbf{i}_\textbf{m}}$$

### Step 5: Fit the results

Repeat Steps 1 through 4 for different values of $m$ and fit the
Fit the results for the averaged sequence delity to the model:
$$ \textit{F}_g(m,|\psi \rangle ) = A_0 p^m +B_0$$
where $A_0$ and $B_0$ absorb state preparation and measurement errors as well as an edge effect from the
error on the final gate.

$p$ determines the average error-rate $r$, which is also called **Error per Clifford (EPC)** 
according to the relation $$ r = 1-p-\frac{1-p}{2^n} = \frac{2^n-1}{2^n}(1-p)$$

In [16]:
#Create the RB fitter
rb_fit = rb.RBFitter(None, xdata, rb_opts['rb_pattern'])
for rb_seed,rb_circ_seed in enumerate(rb_circs):
    qobj = qiskit.compile(rb_circ_seed,
    backend=backend,
    basis_gates=basis_gates)
    job = backend.run(qobj, noise_model=noise_model)

    #add data to the fitter
    rb_fit.add_data(job.result())
    print('After seed %d, EPC %f'%(rb_seed,rb_fit.fit[0]['epc']))

After seed 0, EPC 0.011154
After seed 1, EPC 0.011494
After seed 2, EPC 0.010545
After seed 3, EPC 0.011478
After seed 4, EPC 0.011082
